In [1]:
%load_ext autoreload
%autoreload 2
from myModel import *


Using TensorFlow backend.


## Initialize the GPU

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

## Load the images IDs and their respective labels

In [3]:
id_list = []
for file in os.listdir(FOLDER_IMG):
    if file[-4:] == ".png":
        id = file[:-4]
        id_list.append(id)
        img = cv2.imread(FOLDER_IMG + file, 0)
        img_shape = img.shape
        
my_labs = {}
with open(LABEL_FILE) as f:
    content = f.readlines()
    for line in content:
        line = line.replace('\n', '')
        id = line.split(' ')[0]
        lab = line.split(' ')[1]
        my_labs[id] = lab

print("There are " + str(len(id_list)) + " words in total.")

There are 175280 words in total.


## Load the model

In [4]:
model, y_pred, inputs = get_Model(True)

ada = Adadelta()
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)


In [5]:
model.load_weights(OUTPUT_DIR +"weights.h5")

## Train

In [6]:
if len(check_alphabet(alphabet, my_labs)) == 0:
    print("All letters in the labels are contained in the alphabet.")
else:
    print("The letters " + str(check_alphabet(alphabet, my_labs))+ " are in the labels but not in the alphabet.")

All letters in the labels are contained in the alphabet.


In [7]:
for i in range(100):
    generator = TextImageGenerator(id_list, my_labs, WIDTH, HEIGHT, BATCH_SIZE, DOWNSAMPLE_FACTOR, NB_MAX_IMAGES)
    generator.build_data()

    run_name = datetime.datetime.now().strftime('%Y:%m:%d:%H:%M:%S')
    test_func = K.function([inputs], [y_pred])  
    viz_cb = VizCallback(run_name, test_func, generator.next_batch_val(), min(6, BATCH_SIZE))
    
    model.fit_generator(generator=generator.next_batch_train(),
                        steps_per_epoch=int(generator.n_train / BATCH_SIZE),
                        epochs=50,
                        callbacks=[viz_cb],
                        validation_data=generator.next_batch_val(),
                        validation_steps=int(generator.n_val / BATCH_SIZE))
    
    del generator
    del viz_cb
    gc.collect()

12800  Image Loading start...
 Image Loading finish...
Epoch 1/50
200/200 [==============================] - 229s 1s/step - loss: 0.1813 - val_loss: 0.1252

Out of 256 samples:  Mean edit distance:0.047 Mean normalized edit distance: 0.006
Epoch 2/50
200/200 [==============================] - 221s 1s/step - loss: 0.0291 - val_loss: 0.0597

Out of 256 samples:  Mean edit distance:0.016 Mean normalized edit distance: 0.002
Epoch 3/50
200/200 [==============================] - 225s 1s/step - loss: 0.0080 - val_loss: 0.0523

Out of 256 samples:  Mean edit distance:0.020 Mean normalized edit distance: 0.003
Epoch 4/50
200/200 [==============================] - 220s 1s/step - loss: 0.0031 - val_loss: 0.0468

Out of 256 samples:  Mean edit distance:0.012 Mean normalized edit distance: 0.005
Epoch 5/50
200/200 [==============================] - 220s 1s/step - loss: 0.0023 - val_loss: 0.0430

Out of 256 samples:  Mean edit distance:0.016 Mean normalized edit distance: 0.002
Epoch 6/50
200/200 [

KeyboardInterrupt: 

# Save the weights

In [8]:
folder = "../"
model.save_weights(folder + "weights_crnn.h5.h5") 
model.save_weights(OUTPUT_DIR +"weights.h5")